## MySQL

### DataFrame covid-19-bogota.csv

In [26]:
import pandas as pd

In [27]:
df_cv19bog = pd.read_csv('covid-19-bogota.csv', encoding = 'latin-1', sep = ';').dropna()

### PyMySQL

#### Crear base de datos

In [38]:
from secrets import mysql_info
import pymysql.cursors

db_covid_name = 'covid19'

In [30]:
myconnect = pymysql.connect(
    host = mysql_info['host'],
    user = mysql_info['user'],
    password = mysql_info['password']
)

In [31]:
try:
    with myconnect.cursor() as cursor:
        cursor.execute('CREATE DATABASE {db}'.format(db=db_covid_name))
        cursor.execute("SHOW DATABASES")
        for x in cursor:
            print(x)
finally:
    myconnect.close()

('information_schema',)
('covid19',)
('mysql',)
('performance_schema',)
('sys',)


### Crear tabla localidades

In [168]:
table_localities_name = 'localities'

In [169]:
local_fields = pd.DataFrame([
    {'name':'id', 'type':'INT'},
    {'name':'name_wiki', 'type':'TEXT'},
    {'name':'name_covid', 'type':'TEXT'}
])
local_fields['concat'] = local_fields['name'] + ' ' + local_fields['type']
local_fields

,name,type,concat
0,id,INT,id INT
1,name_wiki,TEXT,name_wiki TEXT
2,name_covid,TEXT,name_covid TEXT


In [170]:
mess_fields = ','.join(local_fields['concat'].tolist())
mess_fields

'id INT,name_wiki TEXT,name_covid TEXT'

In [171]:
sqlQuery = '''CREATE TABLE IF NOT EXISTS {tb_name}({fields})'''.format(tb_name=table_localities_name, fields=mess_fields)
sqlQuery

'CREATE TABLE IF NOT EXISTS localities(id INT,name_wiki TEXT,name_covid TEXT)'

In [172]:
myconnect = pymysql.connect(
    host = mysql_info['host'],
    user = mysql_info['user'],
    password = mysql_info['password'],
    db = db_covid_name,
    cursorclass=pymysql.cursors.DictCursor
)

In [173]:
try:
    with myconnect.cursor() as cursor:
        cursor.execute(sqlQuery)
        for x in cursor:
            print(x)
finally:
    myconnect.close()

### Insertar localidades

In [174]:
df_cv19bog.columns

Index(['Fecha de inicio de síntomas', 'Fecha de diagnóstico',
       'Ciudad de residencia', 'Localidad de residencia', 'Edad',
       'Unidad de medida de la edad', 'Sexo', 'Tipo de caso', 'Ubicación',
       'Estado'],
      dtype='object')

In [175]:
localities_covid19 = pd.DataFrame(df_cv19bog['Localidad de residencia'].unique(),columns=['localidad'])

In [176]:
localities_wiki = pd.DataFrame([
    {'id':1, 'name': 'Usaquén'},
    {'id':2, 'name': 'Chapinero'},
    {'id':3, 'name': 'Santa Fe'},
    {'id':4, 'name': 'San Cristóbal'},
    {'id':5, 'name': 'Usme'},
    {'id':6, 'name': 'Tunjuelito'},
    {'id':7, 'name': 'Bosa'},
    {'id':8, 'name':  'Kennedy'},
    {'id':9, 'name': 'Fontibón'},
    {'id':10, 'name': 'Engativá'},
    {'id':11, 'name': 'Suba'},
    {'id':12, 'name': 'Barrios Unidos'},
    {'id':13, 'name': 'Teusaquillo'},
    {'id':14, 'name': 'Los Mártires'},
    {'id':15, 'name': 'Antonio Nariño'},
    {'id':16, 'name': 'Puente Aranda'},
    {'id':17, 'name': 'La Candelaria'},
    {'id':18, 'name': 'Rafael Uribe Uribe'},
    {'id':19, 'name': 'Ciudad Bolívar'},
    {'id':20, 'name': 'Sumapaz'}
])

In [177]:
df_local = pd.merge(localities_wiki,localities_covid19,left_on='name',right_on='localidad',how='outer')
df_local

,id,name,localidad
0,1.0,Usaquén,Usaquén
1,2.0,Chapinero,Chapinero
2,3.0,Santa Fe,Santa Fe
3,4.0,San Cristóbal,San Cristóbal
4,5.0,Usme,Usme
5,6.0,Tunjuelito,Tunjuelito
6,7.0,Bosa,Bosa
7,8.0,Kennedy,Kennedy
8,9.0,Fontibón,Fontibón
9,10.0,Engativá,Engativá


In [178]:
df_local.rename(columns={'id': local_fields['name'][0],'name': local_fields['name'][1], 'localidad': local_fields['name'][2]})
df_local.dropna(inplace= True,subset=[local_fields['name'][0]])
df_local

,id,name,localidad
0,1.0,Usaquén,Usaquén
1,2.0,Chapinero,Chapinero
2,3.0,Santa Fe,Santa Fe
3,4.0,San Cristóbal,San Cristóbal
4,5.0,Usme,Usme
5,6.0,Tunjuelito,Tunjuelito
6,7.0,Bosa,Bosa
7,8.0,Kennedy,Kennedy
8,9.0,Fontibón,Fontibón
9,10.0,Engativá,Engativá


In [179]:
insert_fields = ','.join(local_fields['name'].tolist())
insert_fields

'id,name_wiki,name_covid'

In [180]:
df_insert_values = df_local.applymap(lambda x: "'" + str(x) + "'")
df_insert_values['insert'] = "(" + df_insert_values.apply(",".join, axis=1) + ")"
insert_values = ",".join(df_insert_values['insert'].to_list())
insert_values

"('1.0','Usaquén','Usaquén'),('2.0','Chapinero','Chapinero'),('3.0','Santa Fe','Santa Fe'),('4.0','San Cristóbal','San Cristóbal'),('5.0','Usme','Usme'),('6.0','Tunjuelito','Tunjuelito'),('7.0','Bosa','Bosa'),('8.0','Kennedy','Kennedy'),('9.0','Fontibón','Fontibón'),('10.0','Engativá','Engativá'),('11.0','Suba','Suba'),('12.0','Barrios Unidos','Barrios Unidos'),('13.0','Teusaquillo','Teusaquillo'),('14.0','Los Mártires','Los Mártires'),('15.0','Antonio Nariño','Antonio Nariño'),('16.0','Puente Aranda','Puente Aranda'),('17.0','La Candelaria','La Candelaria'),('18.0','Rafael Uribe Uribe','Rafael Uribe Uribe'),('19.0','Ciudad Bolívar','Ciudad Bolívar'),('20.0','Sumapaz','nan')"

In [181]:
sqlQuery = '''INSERT INTO {tb_name} ({fields}) VALUES {values};'''.format(tb_name=table_localities_name,fields=insert_fields,values=insert_values)
sqlQuery

"INSERT INTO localities (id,name_wiki,name_covid) VALUES ('1.0','Usaquén','Usaquén'),('2.0','Chapinero','Chapinero'),('3.0','Santa Fe','Santa Fe'),('4.0','San Cristóbal','San Cristóbal'),('5.0','Usme','Usme'),('6.0','Tunjuelito','Tunjuelito'),('7.0','Bosa','Bosa'),('8.0','Kennedy','Kennedy'),('9.0','Fontibón','Fontibón'),('10.0','Engativá','Engativá'),('11.0','Suba','Suba'),('12.0','Barrios Unidos','Barrios Unidos'),('13.0','Teusaquillo','Teusaquillo'),('14.0','Los Mártires','Los Mártires'),('15.0','Antonio Nariño','Antonio Nariño'),('16.0','Puente Aranda','Puente Aranda'),('17.0','La Candelaria','La Candelaria'),('18.0','Rafael Uribe Uribe','Rafael Uribe Uribe'),('19.0','Ciudad Bolívar','Ciudad Bolívar'),('20.0','Sumapaz','nan');"

In [185]:
myconnect = pymysql.connect(
    host = mysql_info['host'],
    user = mysql_info['user'],
    password = mysql_info['password'],
    db = db_covid_name,
    cursorclass=pymysql.cursors.DictCursor
)

In [186]:
cursor = myconnect.cursor()

In [187]:
cursor.execute(sqlQuery)

20

In [188]:
for x in cursor:
    print(x)

In [184]:
#try:
with myconnect.cursor() as cursor:
    cursor.execute(sqlQuery)
#finally:
myconnect.close()

### Crear tabla covid-19-bogota desde DataFrame

In [32]:
from sqlalchemy import create_engine
table_covid_name='cov19bogorig'

In [33]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user=mysql_info['user'],
                               pw=mysql_info['password'],
                               host=mysql_info['host'],
                               db=db_covid_name))

In [34]:
myconnect = engine.connect()

In [35]:
df_cv19bog.to_sql(table_covid_name, con=myconnect, if_exists='replace');

### MySQL Connector

In [28]:
mydb = mysql.connector.connect(
    host = mysql_info['host'],
    user = mysql_info['user'],
    password = mysql_info['password'],
    db = db_name
)

### Crear base de datos

In [30]:
db_name = 'covid19'
mycursor = mydb.cursor()
mycursor.execute('CREATE DATABASE '+ db_name)

In [31]:
mycursor.execute("SHOW DATABASES")

In [32]:
for x in mycursor:
    print(x)

('information_schema',)
('covid19',)
('mysql',)
('performance_schema',)
('sys',)


In [33]:
mydb_covid19 = mysql.connector.connect(
    host=mysql_info['host'],
    database=db_name,
    user=mysql_info['user'],
    password=mysql_info['password'])

In [34]:
mydb_covid19

### Conexión con MySQL desde Pandas